In [1]:
import datetime as dt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import math
import netCDF4


In [2]:
## first, lets set some info about the times, level, etc., we want to plot

lev2plot = 500           #


year = 2019          
month = 2
day = 1
total_days = 1           # how many days are we plotting?
first_fhr = 0            # setting to 0 = 00 UTC, 6 = 06 UTC, ...
hours = total_days * 24  # total hours in time range to plot
hr_inc = 6               # the delta time for the data

date1 = dt.datetime(year,month,day,0)  # first date to plot

#create an array of times we want to plot...
times = [date1 + dt.timedelta(hours=x) for x in range(0,hours,hr_inc)]
#...and put them in the correct time units "hours since 00 UTC 1 Jan 1800 "
date_list = netCDF4.date2num(times,units="hours since 1800-01-01 00:00:00",calendar="gregorian") #change dates to netcdf times


In [3]:
# Lets open our data file and get out data
# data is in /erai/YYYY/VAR.nc
# climo data is in /erai/climo/mean/VAR.nc and /erai/climo/stdev/VAR.nc


gfile = xr.open_dataset("/erai/"+str(year)+"/pt_pv."+str(year)+".nc")  ##Change to your data/file
g_var = gfile["pt_pv"]
g_data  = (g_var.loc[dict(lat=slice(0,90))]) ##


pfile = xr.open_dataset("/erai/"+str(year)+"/pmsl."+str(year)+".nc")  ##Change to your data/file
p_var = pfile["pmsl"]
p_data  = (p_var.loc[dict(lat=slice(0,90) )]) / 100. ##


lat = g_data["lat"].values
lon = g_data["lon"].values

In [4]:
for date in date_list: #loop through each day 
    ind = np.where(date_list == date)[0]
    for d in ind:
        pt_dt = g_data.loc[dict(time=times[d] )]
        pmsl = p_data.loc[dict(time=times[d] )]
        print(pt_dt.coords)
        
        print("Date "+times[d].strftime("%H UTC %d %b %Y"))
        valid_label = (times[d].strftime("Valid: %H UTC %d %b %Y"))

        plt.Figure(figsize=(17,17),dpi=170)    ## <---Set fig size here!

        #Set plot as an orthographic projection looking down at the Earth from the pole with 0˚ at 6-oclock
        ax = plt.axes(projection=ccrs.Orthographic(-90,90))  ## <-- (lon and lat for map center)   
        ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='dimgray',facecolor='none', linewidth=0.7)
        ax.outline_patch.set_edgecolor('none')
        gl = ax.gridlines(color="grey",linestyle=":",linewidth=0.5)  
        glevs = range(950,1080,10)  ## <---set contour interval for geo. heights

        [x,y] = np.meshgrid(lon,lat)  ## <--- set lat/lon as coords on grid
        
        plt.title("Dynamic Tropopause Potential Temp & PMSL \n"+valid_label)
        p1plot = ax.contour(x,y,pmsl,glevs,transform=ccrs.PlateCarree(),extend='both',colors='black',linewidth=1)
        ax.clabel(p1plot, inline=True, fmt="%.0f", fontsize=7)
        
        cmap = matplotlib.cm.plasma  ##<--- color map to use
        norm = matplotlib.colors.Normalize(vmin=250, vmax=450)    ##<---min/max for PMSL

        p2plot = ax.contourf(x,y,pt_dt,100,transform=ccrs.PlateCarree(),cmap=cmap, alpha=0.5)
        plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), label="Potential Temperature (K)", alpha=.5)
        
        plot_label = times[d].strftime("%Y%m%d%H")
        plt.savefig(f"{plot_label}_DTpmsl.png",format='png')
        plt.clf()
        

Coordinates:
    time     datetime64[ns] 2019-02-01
  * lat      (lat) float32 0.35087654 1.0526296 1.7543826 ... 88.76695 89.46282
  * lon      (lon) float32 0.0 0.703125 1.40625 ... 358.59375 359.29688
Date 00 UTC 01 Feb 2019


/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.
  app.launch_new_instance()
/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1478: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Coordinates:
    time     datetime64[ns] 2019-02-01T06:00:00
  * lat      (lat) float32 0.35087654 1.0526296 1.7543826 ... 88.76695 89.46282
  * lon      (lon) float32 0.0 0.703125 1.40625 ... 358.59375 359.29688
Date 06 UTC 01 Feb 2019
Coordinates:
    time     datetime64[ns] 2019-02-01T12:00:00
  * lat      (lat) float32 0.35087654 1.0526296 1.7543826 ... 88.76695 89.46282
  * lon      (lon) float32 0.0 0.703125 1.40625 ... 358.59375 359.29688
Date 12 UTC 01 Feb 2019
Coordinates:
    time     datetime64[ns] 2019-02-01T18:00:00
  * lat      (lat) float32 0.35087654 1.0526296 1.7543826 ... 88.76695 89.46282
  * lon      (lon) float32 0.0 0.703125 1.40625 ... 358.59375 359.29688
Date 18 UTC 01 Feb 2019
